In [1]:
import librosa
import librosa.display
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import glob
import torchaudio

In [2]:
import generate_csv
import data
import train
import model

In [3]:
data_config = data.DatasetConfig()
#generate_csv.gen_csv(["lofi", "classical"], data_config)

In [7]:
df = pd.read_csv("out.csv")
df["class"] = data.encode_classes(df["class"])
size = len(df)
all_indices = np.arange(size)
np.random.shuffle(all_indices)
df = np.array(df)

# Model hyperparameters
EPOCHS = 30
DIMS = 64
N_LAYERS = [4,2,2]
DIRECTIONS = [2]
LEARN_RATE = 1e-5
TRAIN_SIZE = 0.8
VALID_SIZE = 0.1

tr_dataset = data.MusicDataset(df[all_indices][:int(size*TRAIN_SIZE)], data_config)
va_dataset = data.MusicDataset(df[all_indices][int(size*TRAIN_SIZE):int(size*(TRAIN_SIZE+VALID_SIZE))], data_config)
te_dataset = data.MusicDataset(df[all_indices][int(size*TRAIN_SIZE+VALID_SIZE):], data_config)
#elem = tr_dataset[1]
#print([e.shape for e in elem[:-1]], elem[-1])
#print("calculated size", tr_dataset.num_mels, tr_dataset.time_steps)
#quit()
tr_load = DataLoader(tr_dataset, batch_size=None, batch_sampler=None, shuffle=False, num_workers=0)
va_load = DataLoader(va_dataset, batch_size=None, batch_sampler=None, shuffle=False, num_workers=0)
te_load = DataLoader(te_dataset, batch_size=None, batch_sampler=None, shuffle=False, num_workers=0)

network = model.MelNet(DIMS, N_LAYERS, 2, DIRECTIONS, data_config)
try:
    network.load(glob.glob("model_checkpoints/*.model")[-1])
except IndexError:
    pass

Trainable Parameters: 1.192M
